In [52]:
import numpy as np
import pandas as pd
from itertools import combinations
from itertools import product

In [53]:
df = pd.read_csv('../data/train_and_test2.csv')

In [54]:
df.head(10)

,Passengerid,Age,Fare,Sex,sibsp,zero,zero.1,zero.2,zero.3,zero.4,...,zero.12,zero.13,zero.14,Pclass,zero.15,zero.16,Embarked,zero.17,zero.18,2urvived
0,1,22.0,7.2500,0,1,0,0,0,0,0,...,0,0,0,3,0,0,2.0,0,0,0
1,2,38.0,71.2833,1,1,0,0,0,0,0,...,0,0,0,1,0,0,0.0,0,0,1
2,3,26.0,7.9250,1,0,0,0,0,0,0,...,0,0,0,3,0,0,2.0,0,0,1
3,4,35.0,53.1000,1,1,0,0,0,0,0,...,0,0,0,1,0,0,2.0,0,0,1
4,5,35.0,8.0500,0,0,0,0,0,0,0,...,0,0,0,3,0,0,2.0,0,0,0
5,6,28.0,8.4583,0,0,0,0,0,0,0,...,0,0,0,3,0,0,1.0,0,0,0
6,7,54.0,51.8625,0,0,0,0,0,0,0,...,0,0,0,1,0,0,2.0,0,0,0
7,8,2.0,21.0750,0,3,0,0,0,0,0,...,0,0,0,3,0,0,2.0,0,0,0
8,9,27.0,11.1333,1,0,0,0,0,0,0,...,0,0,0,3,0,0,2.0,0,0,1
9,10,14.0,30.0708,1,1,0,0,0,0,0,...,0,0,0,2,0,0,0.0,0,0,1


In [55]:
# PassengerId を削除
df = df.drop(columns=["Passengerid"], errors='ignore')

# 全ての値が0の列を削除
df = df.loc[:, (df != 0).any(axis=0)]


In [56]:
df.head(10)

,Age,Fare,Sex,sibsp,Parch,Pclass,Embarked,2urvived
0,22.0,7.2500,0,1,0,3,2.0,0
1,38.0,71.2833,1,1,0,1,0.0,1
2,26.0,7.9250,1,0,0,3,2.0,1
3,35.0,53.1000,1,1,0,1,2.0,1
4,35.0,8.0500,0,0,0,3,2.0,0
5,28.0,8.4583,0,0,0,3,1.0,0
6,54.0,51.8625,0,0,0,1,2.0,0
7,2.0,21.0750,0,3,1,3,2.0,0
8,27.0,11.1333,1,0,2,3,2.0,1
9,14.0,30.0708,1,1,0,2,0.0,1


In [57]:
# 2値データ以外を抽出
df_numeric = df[['Age', 'Fare', 'sibsp', 'Parch','Pclass','Embarked']]

# 平均値の計算
column_means = df_numeric.mean()

# 平均値を表示
print("各数値列の平均値：")
display(column_means)



各数値列の平均値：


Age         29.503186
Fare        33.281086
sibsp        0.498854
Parch        0.385027
Pclass       2.294882
Embarked     1.492731
dtype: float64

In [58]:
# 2値データ以外を抽出
numeric_cols = df_numeric.columns

# 各数値列ごとに、平均以上を1、未満を0として上書き
for col in numeric_cols:
    mean_val = df[col].mean()
    df[col] = (df[col] >= mean_val).astype(int)



In [59]:
df.head()

,Age,Fare,Sex,sibsp,Parch,Pclass,Embarked,2urvived
0,0,0,0,1,0,1,1,0
1,1,1,1,1,0,0,0,1
2,0,0,1,0,0,1,1,1
3,1,1,1,1,0,0,1,1
4,1,0,0,0,0,1,1,0


In [60]:
# Survivedを削除
train_df = df.drop(columns=["2urvived"], errors='ignore')


In [61]:
df.to_csv('row_data.csv')
train_df.to_csv('train_data.csv')

In [62]:
# データ読み込み
data_np = train_df.values
n, p = data_np.shape

# 全2^pの候補ベクトル生成
candidate = np.array([list(map(int, format(i, f'0{p}b'))) for i in range(2**p)])

# 出現頻度（確率ベクトル）
countvec = np.zeros(2**p)
for i in range(n):
    for j in range(2**p):
        if np.array_equal(data_np[i], candidate[j]):
            countvec[j] += 1
probvec = countvec / n

# ハミング距離関数
def hamming(x, y):
    return np.sum(x != y)

# 目的関数（2点に対して）
def objective(c1, c2):
    total = 0
    for i in range(2**p):
        if probvec[i] > 0:
            d1 = hamming(candidate[i], c1)
            d2 = hamming(candidate[i], c2)
            total += probvec[i] * min(d1, d2)
    return total

# 全探索：すべての組合せに対して評価
best_score = float('inf')
best_pair = None
for i, j in combinations(range(2**p), 2):
    score = objective(candidate[i], candidate[j])
    if score < best_score:
        best_score = score
        best_pair = (i, j)

# 出力
best_c1 = candidate[best_pair[0]]
best_c2 = candidate[best_pair[1]]

print("=== 全探索による最適解 ===")
print("代表点1:", best_c1.tolist())
print("代表点2:", best_c2.tolist())
print("目的関数値:", best_score)

# 結果を辞書形式で準備
result = {
    "代表点1": [best_c1.tolist()],
    "代表点2": [best_c2.tolist()],
    "目的関数値": [best_score]
}

# データフレームとして出力
result_df = pd.DataFrame(result)

# CSVに保存
result_df.to_csv("principal_points_best_result.csv", index=False)

print("結果を 'principal_points_best_result.csv' に出力しました。")



=== 全探索による最適解 ===
代表点1: [0, 0, 0, 0, 0, 1, 1]
代表点2: [1, 1, 1, 1, 0, 0, 1]
目的関数値: 1.6569900687547738
結果を 'principal_points_best_result.csv' に出力しました。


In [63]:
# === 1. 全体データ（Survived付き）を読み込み ===
df = df.loc[:, ~df.columns.str.contains("^Unnamed")]

# === 2. 特徴量と目的変数に分離 ===
features = df.iloc[:, :-1].astype(int)  # int型に変換して比較可能にする
survived = df.iloc[:, -1]
X = features.values

# === 3. 全探索で得られた代表点（Principal Points） ===
pp1 = np.array([0, 0, 0, 0, 0, 1, 1])
pp2 = np.array([1, 1, 1, 1, 0, 0, 1])

# === 4. グループ割当関数（ハミング距離による） ===
def assign_to_group(x, pp1, pp2):
    d1 = np.sum(x != pp1)
    d2 = np.sum(x != pp2)
    return 0 if d1 <= d2 else 1

# === 5. 各データを代表点に割り当てる ===
assigned = np.array([assign_to_group(x, pp1, pp2) for x in X])

# === 6. 割当と生存ラベルを対応付けて集計 ===
df_compare = pd.DataFrame({
    "AssignedGroup": assigned,
    "Survived": survived
})

group_summary = df_compare.groupby("AssignedGroup")["Survived"].agg(["mean", "count"]).reset_index()
group_summary.rename(columns={"mean": "SurvivalRate", "count": "Count"}, inplace=True)

# === 7. CSV出力 ===
group_summary.to_csv("principal_group_survival_summary.csv", index=False)

# === 8. 結果出力 ===
print("=== 代表点による分類と生存率 ===")
print(group_summary)


=== 代表点による分類と生存率 ===
   AssignedGroup  SurvivalRate  Count
0              0      0.186512    949
1              1      0.458333    360


In [64]:
data_np = train_df.values
n, p = data_np.shape

# 候補ベクトル生成（2^p 個）
candidate = np.array([list(map(int, format(i, f'0{p}b'))) for i in range(2**p)])

# 出現頻度ベクトル（確率ベクトル）
countvec = np.zeros(2**p)
for i in range(n):
    for j in range(2**p):
        if np.array_equal(data_np[i], candidate[j]):
            countvec[j] += 1
probvec = countvec / n

# ハミング距離関数
def hamming(x, y):
    return np.sum(x != y)

# 目的関数
def objective(c1, c2):
    total = 0
    for i in range(2**p):
        if probvec[i] > 0:
            d1 = hamming(candidate[i], c1)
            d2 = hamming(candidate[i], c2)
            total += probvec[i] * min(d1, d2)
    return total

# Step 1: 最頻出点を1つ目の代表点とする
first_idx = np.argmax(probvec)
c1 = candidate[first_idx]

# Step 2: 最も距離が大きい点を2つ目の代表点とする
max_dist = -1
second_idx = -1
for i in range(2**p):
    if i == first_idx:
        continue
    dist = hamming(c1, candidate[i])
    if dist > max_dist:
        max_dist = dist
        second_idx = i
c2 = candidate[second_idx]

# 評価
greedy_score = objective(c1, c2)

# 出力
print("=== 貪欲法による近似解 ===")
print("代表点1:", c1.tolist())
print("代表点2:", c2.tolist())
print("目的関数値:", greedy_score)

# 結果を辞書形式で準備
result = {
    "代表点1": [best_c1.tolist()],
    "代表点2": [best_c2.tolist()],
    "目的関数値": [best_score]
}

# データフレームとして出力
result_df = pd.DataFrame(result)

# CSVに保存
result_df.to_csv("principal_points_best_result.csv", index=False)

print("結果を 'principal_points_best_result.csv' に出力しました。")


=== 貪欲法による近似解 ===
代表点1: [0, 0, 0, 0, 0, 1, 1]
代表点2: [1, 1, 1, 1, 1, 0, 0]
目的関数値: 1.7211611917494258
結果を 'principal_points_best_result.csv' に出力しました。


In [ ]:
# === 1. 全体データ（Survived付き）を読み込み ===
df = df.loc[:, ~df.columns.str.contains("^Unnamed")]

# === 2. 特徴量と目的変数に分離 ===
features = df.iloc[:, :-1].astype(int)  # int型に変換して比較可能にする
survived = df.iloc[:, -1]
X = features.values

# === 3. 全探索で得られた代表点（Principal Points） ===
pp1 = np.array([0, 0, 0, 0, 0, 1, 1])
pp2 = np.array([1, 1, 1, 1, 1, 0, 0,])

# === 4. グループ割当関数（ハミング距離による） ===
def assign_to_group(x, pp1, pp2):
    d1 = np.sum(x != pp1)
    d2 = np.sum(x != pp2)
    return 0 if d1 <= d2 else 1

# === 5. 各データを代表点に割り当てる ===
assigned = np.array([assign_to_group(x, pp1, pp2) for x in X])

# === 6. 割当と生存ラベルを対応付けて集計 ===
df_compare = pd.DataFrame({
    "AssignedGroup": assigned,
    "Survived": survived
})

group_summary = df_compare.groupby("AssignedGroup")["Survived"].agg(["mean", "count"]).reset_index()
group_summary.rename(columns={"mean": "SurvivalRate", "count": "Count"}, inplace=True)

# === 7. CSV出力 ===
group_summary.to_csv("principal_group_survival_summary.csv", index=False)

# === 8. 結果出力 ===
print("=== 代表点による分類と生存率 ===")
print(group_summary)


=== 代表点による分類と生存率 ===
   AssignedGroup  SurvivalRate  Count
0              0      0.200198   1009
1              1      0.466667    300
